In [11]:
from new_model import *
from tqdm import tqdm

In [12]:
seed = 777
random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## 1. CIFAR 10 데이터셋 다운로드

In [3]:
#Data 다운로드 경로지정
DATA_ROOT = "../data_cifar10"
batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## 2. CIFAR 100 데이터셋 다운로드(참고)

In [13]:
#Data 다운로드 경로지정
DATA_ROOT = "../data_cifar100"
batch_size = 128

#Data Process
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

train_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR100) * 95 / 100)
num_val = len(train_CIFAR100) - num_train
train_CIFAR100, val_CIFAR100 = torch.utils.data.random_split(train_CIFAR100, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR100, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## 4. train_model과 test_model 저장

In [14]:
def train_model(model, mode):    
    # total number of training epochs
    EPOCHS = 200
    CHECKPOINT_PATH = "./saved_model"
    
    best_val_acc = 0
    current_learning_rate = INITIAL_LR

    print("==> Training starts!")

    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [60, 120, 160], gamma = 0.2)
    
    for i in tqdm(range(0, EPOCHS)):
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0
        train_loss = 0

        
        '''
        train loop
        '''
        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()
            


            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)
        
        # scheduler
        scheduler.step()
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

        
                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc :
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))
        
        if i % 10 == 9 :  #epoch를 5개씩 출력해서 봐보겠다
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss_tr, avg_acc_tr)) 
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
        time.sleep(0.1)
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                                    

    return model #결국 history 변수를 지정해서 epoch 1개씩 loss와 accuracy값을 저장하려고 return 해야됨

In [15]:
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

In [16]:
INITIAL_LR = 0.1

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 5e-4

# Define loss
criterion = nn.CrossEntropyLoss()

## 5. Wide_resnet model을 train 시켜보자
- net : 28x1, 28x2, 28x4, 28x10

## CIFAR10

In [7]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
model = Wide_ResNet(28, 4, 0.3, 10).to(device)
print("The model is deployed to", device)

| Wide-Resnet 28x4
The model is deployed to cuda:2


In [8]:
train_model(model, "resnet28x4_cifar10")

==> Training starts!


  5%|▌         | 10/200 [08:45<2:48:24, 53.18s/it]

Epoch 10: progress time is 524.96 sec
Training loss: 0.5029, Training accuracy: 0.8267
Validation loss: 0.5284, Validation accuracy: 0.8128


 10%|█         | 20/200 [17:39<2:40:32, 53.51s/it]

Epoch 20: progress time is 1059.68 sec
Training loss: 0.3727, Training accuracy: 0.8727
Validation loss: 0.6748, Validation accuracy: 0.7832


...

 95%|█████████▌| 190/200 [2:50:45<09:06, 54.70s/it]

Epoch 190: progress time is 10245.63 sec
Training loss: 0.0030, Training accuracy: 0.9999
Validation loss: 0.1648, Validation accuracy: 0.9600


100%|██████████| 200/200 [2:59:43<00:00, 53.92s/it]

Epoch 200: progress time is 10783.06 sec
Training loss: 0.0024, Training accuracy: 1.0000
Validation loss: 0.1550, Validation accuracy: 0.9548
==> Optimization finished! Best validation accuracy: 0.9640


Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

## CIFAR100

In [17]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
model = Wide_ResNet(28, 4, 0.3, 100).to(device)
print("The model is deployed to", device)

| Wide-Resnet 28x4
The model is deployed to cuda:2


In [18]:
train_model(model, "resnet28x4_cifar100")

==> Training starts!


  5%|▌         | 10/200 [08:50<2:49:59, 53.68s/it]

Epoch 10: progress time is 530.79 sec
Training loss: 1.6058, Training accuracy: 0.5504
Validation loss: 2.4850, Validation accuracy: 0.3828


 10%|█         | 20/200 [17:45<2:39:54, 53.30s/it]

Epoch 20: progress time is 1065.69 sec
Training loss: 1.2012, Training accuracy: 0.6512
Validation loss: 1.6015, Validation accuracy: 0.5652


...

 95%|█████████▌| 190/200 [2:52:07<09:16, 55.63s/it]

Epoch 190: progress time is 10327.78 sec
Training loss: 0.0116, Training accuracy: 0.9997
Validation loss: 0.8517, Validation accuracy: 0.7772


100%|██████████| 200/200 [3:01:12<00:00, 54.36s/it]

Epoch 200: progress time is 10872.7 sec
Training loss: 0.0114, Training accuracy: 0.9997
Validation loss: 0.8877, Validation accuracy: 0.7772
==> Optimization finished! Best validation accuracy: 0.7888


Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3)

In [19]:
send_email('resnet28x4_cifar100 finished')

## test_model을 통해 결과확인

In [10]:
#wide_resnet 28x1의 결과값
checkpoint = torch.load("./saved_model/resnet28x4_cifar10.pth")
model.load_state_dict(checkpoint['state_dict'])
test_model(model)

Total examples is 10000, correct examples is 9538; Test accuracy: 0.9538


In [20]:
#wide_resnet 28x1의 결과값
checkpoint = torch.load("./saved_model/resnet28x4_cifar100.pth")
model.load_state_dict(checkpoint['state_dict'])
test_model(model)

Total examples is 10000, correct examples is 7724; Test accuracy: 0.7724
